In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
from vectorizer import create_vectorizer, create_data, vec_to_categories

In [3]:
data = pd.read_excel('data/combined.xlsx')

corpus = data['text'].values
corpus[pd.isnull(corpus)] = ''
corpus = corpus.tolist()

labels = data['language'].values.tolist()

In [4]:
categories = [
    'slovak',
    'english',
    'french'
]

In [5]:
vectorizer, ngram_counts, vocabulary = create_vectorizer(corpus)

In [6]:
X, Y = create_data(ngram_counts, vocabulary, labels)

MemoryError: Unable to allocate 11.9 GiB for an array with shape (33193, 1, 48151) and data type float64

In [137]:
Y = vec_to_categories(Y)

7586

In [6]:
X[0].shape
len(vocabulary)

48151

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [122]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(26554, 1, 7586)
(6639, 1, 7586)
(26554,)


In [65]:
d = len(vocabulary)

model = tf.keras.models.Sequential([
  tf.keras.Input(shape = (1, d)),
  tf.keras.layers.Dense(1100, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(200, activation='relu'),
  tf.keras.layers.Dense(3)
])

In [66]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [67]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [103]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
830/830 [==============================] - 50s 60ms/step - loss: 0.0513 - accuracy: 0.9799
Epoch 2/5
830/830 [==============================] - 50s 61ms/step - loss: 0.0288 - accuracy: 0.9894
Epoch 3/5
830/830 [==============================] - 53s 64ms/step - loss: 0.0128 - accuracy: 0.9963
Epoch 4/5
830/830 [==============================] - 51s 62ms/step - loss: 0.0123 - accuracy: 0.9961
Epoch 5/5
830/830 [==============================] - 54s 65ms/step - loss: 0.0126 - accuracy: 0.9966


In [104]:
model.evaluate(X_test,  y_test, verbose=2)

208/208 - 3s - loss: 0.5866 - accuracy: 0.9131


[0.5865591764450073, 0.9130893349647522]

In [128]:
def test_string(s, vectorizer):
    V = vectorizer.transform([s]).toarray()
    print(V.shape[1])
    p = model(V.reshape(1, 1, d))
    predictions = tf.nn.softmax(p).numpy()
    print(predictions)
    print(categories[np.argmax(predictions)])
    
s1 = 'Mais les solutions pour y remédier font encore débat à plusieurs égards'
s2 = 'Miroslav Kollár bude kandidovať za predsedu strany Za ľudí'
s3 = 'Coronavirus live news: UK will never know true death toll; Bolsonaro announces negative test'
test_string(s1, vectorizer)
test_string(s2, vectorizer)
test_string(s3, vectorizer)

7586
[[[1.7494536e-37 2.1972864e-39 1.0000000e+00]]]
french
7586
[[[1.000000e+00 4.096280e-39 8.119609e-34]]]
slovak
7586
[[[1.2463016e-06 9.9999726e-01 1.5745266e-06]]]
english


In [110]:
y_train[y_train == 0].shape

(11629,)

In [115]:
model.save('saved_model/full')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/full\assets
